In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from tqdm.auto import tqdm

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans 

In [2]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [3]:
from nltk.corpus import stopwords

In [4]:
from nltk.corpus import stopwords

stop = stopwords.words('indonesian')

In [5]:
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def clean_text(text):
    text = str(text).lower()  # Converts text to lowercase
    text = re.sub('\d+', '', text)  # Removes numbers
    text = re.sub('\[.*?\]', '', text)  # Removes HTML tags
    text = re.sub('https?://\S+|www\.\S+', '', text)  # Removes URLs
    text = re.sub(r"["
                  u"\U0001F600-\U0001F64F"  # Emoticons
                  u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                  u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                  u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                  u"\U00002702-\U000027B0"
                  u"\U000024C2-\U0001F251"
                  "]+", "", text)  # Removes emojis
    text = text.replace('-', ' ')  # Removes hyphens
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Removes punctuations
    ps = PorterStemmer()
    stop = set(stopwords.words('english'))
    text = [ps.stem(word) for word in text.split(' ') if not word in stop]  # Remove stopwords and apply stemming
    text = ' '.join(text)  # Join the words separated by spaces
    return text


In [6]:
## This function to create the corpus which include all sentences in the cleaned text in a list
def make_corpus(text):
    corpus = [''.join(sent) for sent in text]
    return corpus

In [7]:
# urus imbuhan

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# for index, row in df.iterrows():
#     df.loc[index, "clean_content_stemmer"] = stemmer.stem(row["clean_content"])
    

In [8]:
# Save the DataFrame to a CSV file
# df.to_csv('merged2.csv')
df = pd.read_csv('merged2.csv')

In [9]:
df = df[df["content"].notna()]

In [10]:
df = df[df["clean_content_stemmer"].notna()]

In [11]:
df["clean_content_stemmer"].dtype

dtype('O')

In [12]:
corpus = []
for index, row in df.iterrows():
    corpus.append(row["clean_content_stemmer"])

In [13]:
corpus

['menantu jokowi bobbi afif nasut sambang kantor dpw nasdem sumatera utara sumut bobbi sambut ketua dpw nasdem sumut iskandar st sama jumlah penguru meski nama santer sebut',
 'malaysia kembali serang perintah indonesia lewat isu kabut asap belum otorita malaysia juga sempat serang indonesia dengan isu invasi babi hutan ke melakaisu ini mula ketika perintah malaysia akan kirim nota diplomatik kepada indonesia negara tetangga kait kabut asap nota diplomatik itu untuk dorong indonesia agar ambil tindak segera untuk madam bakar lahan yang kirim kabut asap ke wilayah malaysiabaca juga tepi jadi sebab tunggal kabut asap ri akan surati malaysia googletagcmdpushfunct googletagdisplaydiv',
 'orang driver ojek onlin di bekas jadi viral di media sosial pasal driver ojol itu antar pesan makan guna sepeda bukan motor layak ojek pada umumnyadriv ojol itu tahu nama eko susilo detikcom sambang eko di diam di jalan bambu kune rt rw kampung sepat panjang jaya panjang jaya camat rawalumbu kota bekas eko

In [14]:
def exist_dominan(j):
    cnt = 0
    for i in range(0, j):
        z = df[df["cluster_label"] == i].shape[0]
        a = df[(df["cluster_label"] == i) & (df["label_score"] == 0)].shape[0]
        b = df[(df["cluster_label"] == i) & (df["label_score"] == 1)].shape[0]
        if (((a * 100) / z) >= 70):
            cnt += 1
        elif (((b * 100) / z) >= 70):
            cnt += 1
        #print((a * 100) / z , (b * 100) / z)
    if cnt == j:
        return True
    else:
        return False

In [15]:
cluster = []
random_seed = []

tf = TfidfVectorizer()
X = tf.fit_transform(corpus)
#j = true_k
for j in range(2,6):
    for i in tqdm(range(10000), desc="inner loop"):
        model = KMeans(n_clusters=j, init='k-means++', max_iter=1000, n_init=1, random_state=i)

        model.fit(X)
        df["cluster_label"] = model.labels_
        if exist_dominan(j) == True:
            print(f"there are {j} cluster and random seed is {i}")
            cluster.append(j)
            random_seed.append(i)


inner loop:   0%|          | 0/10000 [00:00<?, ?it/s]

inner loop:   0%|          | 0/10000 [00:00<?, ?it/s]

inner loop:   0%|          | 0/10000 [00:00<?, ?it/s]

inner loop:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
# tf = TfidfVectorizer()
# X = tf.fit_transform(corpus)
# for i in range(100):
#     print('random state:', i)
    # true_k = 2
    # model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=1, random_state=i)

#     model.fit(X)

#     order_centroids = model.cluster_centers_.argsort()[:, ::-1]
#     for i in range(true_k):
#         print('Cluster %d:' % i),
#         for ind in order_centroids[i, :10]:
#             print(' %s' % tf.get_feature_names()[ind])
#         print('----------------------')

In [ ]:
# Dari code diatas, jika di un-comment, didapatkan bahwa ada beberapa random state yang menjanjikan, yaitu:
# 51, 61: karena terdapat "mirzani" pada salah satu clusternya.
# 7, 99: karena terdapat "halilintar"
# perhatikan bahwa berita tersebut termasuk berita mengenai gosip artis

In [ ]:
# tf = TfidfVectorizer()
# X = tf.fit_transform(corpus)

# true_k = 2
# model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=1, random_state=99)

# model.fit(X)

# order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# for i in range(true_k):
#     print('Cluster %d:' % i),
#     for ind in order_centroids[i, :10]:
#         print(' %s' % tf.get_feature_names_out()[ind])
#     print('----------------------')

In [ ]:
print(true_k)

In [ ]:
df['Bow Clus Label'] = model.labels_

In [ ]:
print(df['Bow Clus Label'].value_counts())

In [ ]:
print(df[df["Bow Clus Label"] == 1].head(2))


In [ ]:
print(df[df["Bow Clus Label"] == 0].head(2))

In [ ]:
for i in range(0,2):
    for j in range(0,2):
        cnt = df[(df["label_score"] == i) & (df["Bow Clus Label"] == j)].shape[0]
        print(i,j,cnt)

In [ ]:
labels = model.labels_
cluster_center=model.cluster_centers_
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')

In [ ]:
silhouette_score

In [ ]:
df["cluster_label"] = model.labels_

In [ ]:
df.head(10)

In [ ]:
df["cluster_label"].value_counts()

In [ ]:
df.groupby("cluster_label")["label_score"].value_counts()

In [ ]:
def exist_dominan():
    cnt = 0
    for i in range(0,true_k):
        z = df[df["cluster_label"] == i].shape[0]
        a = df[df["cluster_label"] == i & df["label_score"] == 0].shape[0]
        b = df[df["cluster label"] == i & df["label_score"] == 1].shape[0]
        if(((a * 100) / z) >= 75):
            cnt+=1
        elif(((b * 100)/z) >= 75):
            cnt+=1
        
    if(cnt == true_k):
        return True
    else :
        return False


In [ ]:
X

In [ ]:
subset = X[0:5, 0:4]

In [ ]:
print(X[:100, :100])

In [ ]:
print(X)